In [1]:
import io
import subprocess
import os

from tensorflow.compat.v1.keras import layers
import tensorflow.compat.v1 as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from tf_graph_util import convert_variables_to_constants
from lstm import create_lstm

from nmt import create_nmt_models

In [2]:
RANDOM_SEED = 42

In [3]:
import os
def benchmark_model(model):
    if not os.path.exists(f"{model.name}.pbtxt"):
        print("Saving model...")
        tf.keras.backend.clear_session()
        sess = tf.keras.backend.get_session()
#         output_graph_def = tf.graph_util.convert_variables_to_constants(
        output_graph_def = convert_variables_to_constants(
            sess,
            sess.graph.as_graph_def(),
            [node.op.name for node in model.outputs])
        tf.io.write_graph(output_graph_def, './', f'{model.name}.pbtxt')
    else:
        print("Retrieving saved model.")
    
    bench_path = f"{model.name}_benchmark.txt"
    if not os.path.exists(bench_path):
        input_shape = f"1,{','.join(str(dim) for dim in model.input.shape[1:])}"
        print("Running benchmark...")
        cmd = f'../tensorflow/bazel-bin/tensorflow/tools/benchmark/benchmark_model --graph={model.name}.pbtxt --input_layer="{model.input.name}" --input_layer_shape="{input_shape}" --output_layer="{model.output.name}"'
        print(cmd)
        benchmark = subprocess.run([cmd], stderr=subprocess.PIPE, shell=True)
        print("Done.")

        output = benchmark.stderr.decode('unicode_escape')
        split_output = output[output.find('Run Order'):output.find('Top by Computation Time')].split('\n')
        
        with open(bench_path, 'w') as f:
            f.write("\n".join(split_output[1:-2]))
    else:
        print("Retrieving saved benchmark results.")
    
    f = open(bench_path)
    benchmark = pd.read_csv(f, sep="\t").rename(columns=lambda x: x.strip())
    benchmark = benchmark.drop(benchmark.columns[0], axis=1)
    benchmark['name'] = benchmark['[Name]'].apply(lambda x: x.split('/')[0])
    return benchmark

In [4]:
def get_layer_features(model):
    layers = pd.DataFrame()
    layers['name'] = pd.Series([layer.name for layer in model.layers])
    
#     input_dims = {layer.name: [dim.value for dim in layer.input.shape.dims] for layer in model.layers}
    
#     layers['input_shape'] = pd.Series([[dim.value for dim in layer.input.shape.dims] for layer in model.layers])
#     layers['output_shape'] = pd.Series([[dim.value for dim in layer.output.shape.dims] for layer in model.layers])
    layers['input_shape'] = pd.Series([layer.input_shape for layer in model.layers])
    layers['output_shape'] = pd.Series([layer.output_shape for layer in model.layers])

    features = ['units','filters','activation','strides','kernel_size']
    for feature in features:
        layers[feature] = pd.Series(
            [layer.get_config()[feature] if feature in layer.get_config() else None for layer in model.layers])
    return layers

In [5]:
def join_benchmark(features, benchmark):
    speed = benchmark[['name', '[avg ms]']].groupby('name').sum()
    mem = benchmark[['name', '[mem KB]']].groupby('name').max()
    
    return features.join(speed, on='name').join(mem, on='name')

In [6]:

def flatten_shape(shape):
    if not shape:
        return None
    
    def reduce(tup):
        acc = 1
        for val in tup:
            if val:
                acc *= val
        return acc
    
    if isinstance(shape, list):
        return sum(reduce(tup) for tup in shape)
    
    return reduce(shape)


def clean(data):
    cleaned = pd.get_dummies(
        data.dropna(subset=['[avg ms]', '[mem KB]']), columns=['activation'], dummy_na=True)
    

    cleaned['input_size'] = cleaned['input_shape'].apply(flatten_shape)
    cleaned['output_size'] = cleaned['output_shape'].apply(flatten_shape)
    cleaned['stride_size'] = cleaned['strides'].apply(flatten_shape)
    cleaned['kernel_size'] = cleaned['kernel_size'].apply(flatten_shape)

    return cleaned.fillna(-1)



In [7]:
def rf_regression_model(data, column):
    X = data.drop(['name', '[avg ms]', '[mem KB]', 'input_shape', 'output_shape', 'strides'], axis=1)  # Features
    y = data[column]  # Labels

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state = RANDOM_SEED) # 70% training and 30% test

    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 1000, random_state = RANDOM_SEED)
    # Train the model on training data
    rf.fit(X_train, y_train);

    return rf, X_test, y_test

In [8]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

def stacked_regression_model(data, column):

    X = data.drop(['name', '[avg ms]', '[mem KB]', 'input_shape', 'output_shape', 'strides'], axis=1)  # Features
    y = data[column]  # Labels

    estimators = [
        ('rf', RandomForestRegressor(n_estimators=1000, random_state=RANDOM_SEED)),
        ('lr', RidgeCV()),
        ('svr', LinearSVR(random_state=42)),
        
    ]
    reg = StackingRegressor(
        estimators=estimators,
#         final_estimator=RandomForestRegressor(n_estimators=1000, random_state=RANDOM_SEED)
    )
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    reg.fit(X_train, y_train).score(X_test, y_test)
    
    return reg, X_test, y_test

In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

def stacked_regression_model_2(data, column):

    X = data.drop(['name', '[avg ms]', '[mem KB]', 'input_shape', 'output_shape', 'strides'], axis=1)  # Features
    y = data[column]  # Labels
    
    poly = Pipeline([
        ('poly', PolynomialFeatures(degree=3)),
        ('linear', LinearRegression(fit_intercept=False))
    ])

    estimators = [
        ('poly', poly),
        ('lr', RidgeCV()),
        ('svr', LinearSVR(random_state=42)),
        
    ]
    reg = StackingRegressor(
        estimators=estimators,
#         final_estimator=RandomForestRegressor(n_estimators=1000, random_state=RANDOM_SEED)
    )
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    reg.fit(X_train, y_train).score(X_test, y_test)
    
    return reg, X_test, y_test

In [10]:
tf.keras.backend.clear_session() # IMPORTANT for layer names to match up.
model = tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
benchmark = benchmark_model(model)
features = get_layer_features(model)

Retrieving saved model.
Retrieving saved benchmark results.


In [12]:
vgg_df = join_benchmark(features, benchmark)

In [13]:
tf.keras.backend.clear_session() # IMPORTANT for layer names to match up.
tf.keras.backend.set_learning_phase(0)
inception = tf. keras.applications.inception_v3.InceptionV3(
    include_top=True, weights='imagenet')
inception_benchmark = benchmark_model(inception)
inception_features = get_layer_features(inception)
inception_df = join_benchmark(inception_features, inception_benchmark)

Retrieving saved model.
Retrieving saved benchmark results.


In [14]:
# # lstm.summary()

# tf.keras.backend.clear_session() # IMPORTANT for layer names to match up.

# lstm = create_lstm()

# # tf.keras.backend.set_learning_phase(0)
# lstm_benchmark = benchmark_model(lstm)
# lstm_features = get_layer_features(lstm)
# lstm_df = join_benchmark(lstm_features, lstm_benchmark)

In [15]:
data = pd.concat([vgg_df, inception_df])
data.sample(10)

,name,input_shape,output_shape,units,filters,activation,strides,kernel_size,[avg ms],[mem KB]
91,batch_normalization_28,"(None, 35, 35, 96)","(None, 35, 35, 96)",NaN,NaN,None,None,None,0.007,0.000
106,activation_35,"(None, 17, 17, 128)","(None, 17, 17, 128)",NaN,NaN,relu,None,None,0.849,147.968
280,conv2d_89,"(None, 8, 8, 2048)","(None, 8, 8, 448)",NaN,448.0,linear,"(1, 1)","(1, 1)",0.002,0.000
278,activation_84,"(None, 8, 8, 192)","(None, 8, 8, 192)",NaN,NaN,relu,None,None,0.719,49.152
137,batch_normalization_45,"(None, 17, 17, 160)","(None, 17, 17, 160)",NaN,NaN,None,None,None,0.005,0.000
207,activation_61,"(None, 17, 17, 192)","(None, 17, 17, 192)",NaN,NaN,relu,None,None,1.203,221.952
310,mixed10,"[(None, 8, 8, 320), (None, 8, 8, 768), (None, ...","(None, 8, 8, 2048)",NaN,NaN,None,None,None,0.159,524.288
42,batch_normalization_15,"(None, 35, 35, 64)","(None, 35, 35, 64)",NaN,NaN,None,None,None,0.003,0.000
274,batch_normalization_84,"(None, 8, 8, 192)","(None, 8, 8, 192)",NaN,NaN,None,None,None,0.003,0.000
99,max_pooling2d_2,"(None, 35, 35, 288)","(None, 17, 17, 288)",NaN,NaN,None,"(2, 2)",None,0.340,332.928


In [16]:
cleaned = clean(data)

cleaned.head()

,name,input_shape,output_shape,units,filters,strides,kernel_size,[avg ms],[mem KB],activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size
1,block1_conv1,"(None, 224, 224, 3)","(None, 224, 224, 64)",-1.0,64.0,"(1, 1)",9.0,2.657,12845.056,0,1,0,0,150528,3211264,1.0
2,block1_conv2,"(None, 224, 224, 64)","(None, 224, 224, 64)",-1.0,64.0,"(1, 1)",9.0,18.123,12845.056,0,1,0,0,3211264,3211264,1.0
3,block1_pool,"(None, 224, 224, 64)","(None, 112, 112, 64)",-1.0,-1.0,"(2, 2)",-1.0,3.437,3211.264,0,0,0,1,3211264,802816,4.0
4,block2_conv1,"(None, 112, 112, 64)","(None, 112, 112, 128)",-1.0,128.0,"(1, 1)",9.0,7.181,6422.528,0,1,0,0,802816,1605632,1.0
5,block2_conv2,"(None, 112, 112, 128)","(None, 112, 112, 128)",-1.0,128.0,"(1, 1)",9.0,13.004,6422.528,0,1,0,0,1605632,1605632,1.0


In [17]:

X = cleaned.drop(['name', '[avg ms]', '[mem KB]', 'input_shape', 'output_shape', 'strides'], axis=1)  # Features
y = cleaned['[avg ms]']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'avg_ms_actual': y_test, 'avg_ms_pred': y_pred})], axis=1).sample(20)

MSE: 17.122787421224206


,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,avg_ms_actual,avg_ms_pred
228,-1.0,-1.0,-1.0,0,0,0,1,221952,221952,-1.0,0.182,0.236075
89,-1.0,-1.0,-1.0,0,1,0,0,78400,78400,-1.0,0.683,0.920930
197,-1.0,192.0,1.0,1,0,0,0,221952,55488,1.0,0.002,0.002323
111,-1.0,-1.0,-1.0,0,1,0,0,36992,36992,-1.0,0.992,0.882680
164,-1.0,-1.0,-1.0,0,0,0,1,221952,221952,-1.0,0.272,0.236075
158,-1.0,-1.0,-1.0,0,0,0,1,55488,55488,-1.0,0.006,0.005435
190,-1.0,-1.0,-1.0,0,0,0,1,55488,55488,-1.0,0.007,0.005435
212,-1.0,-1.0,-1.0,0,0,0,1,55488,55488,-1.0,0.006,0.005435
214,-1.0,-1.0,-1.0,0,1,0,0,55488,55488,-1.0,0.654,0.911988
162,-1.0,-1.0,-1.0,0,1,0,0,55488,55488,-1.0,0.546,0.911988


In [18]:

X = cleaned.drop(['name', '[avg ms]', '[mem KB]', 'input_shape', 'output_shape', 'strides'], axis=1)  # Features
y = cleaned['[mem KB]']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'mem_KB_actual': y_test, 'mem_KB_pred': y_pred})], axis=1).sample(20)

MSE: 307752.0114859972


,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,mem_KB_actual,mem_KB_pred
111,-1.0,-1.0,-1.0,0,1,0,0,36992,36992,-1.0,147.968,147.710592
291,-1.0,384.0,3.0,1,0,0,0,24576,24576,1.0,0.000,0.000000
222,-1.0,-1.0,-1.0,0,0,0,1,55488,55488,-1.0,0.000,0.000000
68,-1.0,96.0,9.0,1,0,0,0,78400,117600,1.0,0.000,15.138816
218,-1.0,192.0,7.0,1,0,0,0,55488,55488,1.0,0.000,0.000000
25,-1.0,-1.0,-1.0,0,1,0,0,58800,58800,-1.0,235.200,230.245248
242,-1.0,192.0,9.0,1,0,0,0,55488,12288,4.0,0.000,46.583680
209,-1.0,192.0,7.0,1,0,0,0,55488,55488,1.0,0.000,0.000000
178,-1.0,160.0,7.0,1,0,0,0,46240,46240,1.0,0.000,0.000000
76,-1.0,96.0,9.0,1,0,0,0,117600,117600,1.0,0.000,2.031616


In [19]:
# AVG MS RF
rf, X_test, y_test = rf_regression_model(cleaned, '[avg ms]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'avg_ms_actual': y_test, 'avg_ms_pred': y_pred})], axis=1).sample(10)

MSE: 0.34952883344050234


,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,avg_ms_actual,avg_ms_pred
56,-1.0,-1.0,-1.0,0,0,0,1,78400,78400,-1.0,0.004,0.004020
155,-1.0,192.0,1.0,1,0,0,0,221952,55488,1.0,0.002,0.002239
17,-1.0,512.0,9.0,0,1,0,0,100352,100352,1.0,4.547,5.886958
309,-1.0,-1.0,-1.0,0,1,0,0,12288,12288,-1.0,1.287,0.829712
250,-1.0,-1.0,-1.0,0,0,0,1,28672,28672,-1.0,0.005,0.006932
293,-1.0,-1.0,-1.0,0,0,0,1,131072,131072,1.0,0.762,0.484421
42,-1.0,-1.0,-1.0,0,0,0,1,78400,78400,-1.0,0.003,0.004020
52,-1.0,64.0,25.0,1,0,0,0,58800,78400,1.0,0.002,0.004722
7,-1.0,256.0,9.0,0,1,0,0,401408,802816,1.0,7.049,5.958100
39,-1.0,-1.0,-1.0,0,1,0,0,39200,39200,-1.0,0.434,0.799501


In [20]:
# MEM KB RF
rf, X_test, y_test = rf_regression_model(cleaned, '[mem KB]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'mem_kb_actual': y_test, 'mem_kb_pred': y_pred})], axis=1).sample(10)

MSE: 71955.01628587676


,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,mem_kb_actual,mem_kb_pred
229,-1.0,192.0,1.0,1,0,0,0,221952,55488,1.0,0.000,0.000000
1,-1.0,32.0,9.0,1,0,0,0,268203,710432,4.0,0.000,376.104768
151,-1.0,-1.0,-1.0,0,0,0,1,221952,221952,1.0,887.808,889.768704
92,-1.0,-1.0,-1.0,0,1,0,0,117600,117600,-1.0,470.400,463.701504
18,-1.0,-1.0,-1.0,0,0,0,1,100352,25088,4.0,100.352,117.952768
10,-1.0,-1.0,-1.0,0,0,0,1,1382976,341056,4.0,1364.224,1513.208000
123,-1.0,192.0,1.0,1,0,0,0,221952,55488,1.0,0.000,0.000000
21,-1.0,48.0,1.0,1,0,0,0,235200,58800,1.0,0.000,0.000000
232,-1.0,192.0,7.0,1,0,0,0,55488,55488,1.0,0.000,0.000000
288,-1.0,-1.0,-1.0,0,1,0,0,24576,24576,-1.0,98.304,98.304000


In [21]:
# AVG MS stacked
rf, X_test, y_test = stacked_regression_model(cleaned, '[avg ms]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'avg_ms_actual': y_test, 'avg_ms_pred': y_pred})], axis=1).sample(10)

/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


MSE: 2.304815746815876


/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,avg_ms_actual,avg_ms_pred
139,-1.0,160.0,1.0,1,0,0,0,221952,46240,1.0,0.002,0.421680
55,-1.0,-1.0,-1.0,0,0,0,1,78400,78400,-1.0,0.005,0.268668
125,-1.0,-1.0,-1.0,0,0,0,1,55488,55488,-1.0,0.003,0.290258
309,-1.0,-1.0,-1.0,0,1,0,0,12288,12288,-1.0,1.287,0.757708
216,-1.0,192.0,1.0,1,0,0,0,221952,55488,1.0,0.002,0.343088
129,-1.0,-1.0,-1.0,0,1,0,0,55488,55488,-1.0,0.708,0.989789
49,-1.0,-1.0,-1.0,0,1,0,0,117600,117600,-1.0,1.290,1.365286
83,-1.0,-1.0,-1.0,0,1,0,0,78400,78400,-1.0,1.925,0.827638
25,-1.0,-1.0,-1.0,0,1,0,0,58800,58800,-1.0,0.450,0.533482
239,-1.0,-1.0,-1.0,0,1,0,0,55488,55488,-1.0,0.671,0.989789


In [22]:
# AVG MEM stacked
rf, X_test, y_test = stacked_regression_model(cleaned, '[mem KB]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'mem_kb_actual': y_test, 'mem_kb_pred': y_pred})], axis=1).sample(10)

/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


MSE: 1094164.5002207097


/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

,units,filters,kernel_size,activation_linear,activation_relu,activation_softmax,activation_nan,input_size,output_size,stride_size,mem_kb_actual,mem_kb_pred
68,-1.0,96.0,9.0,1,0,0,0,78400,117600,1.0,0.000,22.071346
171,-1.0,160.0,1.0,1,0,0,0,221952,46240,1.0,0.000,-2.551156
214,-1.0,-1.0,-1.0,0,1,0,0,55488,55488,-1.0,221.952,274.605235
39,-1.0,-1.0,-1.0,0,1,0,0,39200,39200,-1.0,156.800,211.032893
80,-1.0,-1.0,-1.0,0,0,0,1,117600,117600,-1.0,0.000,98.741279
303,-1.0,-1.0,-1.0,0,1,0,0,24576,24576,-1.0,98.304,163.962149
76,-1.0,96.0,9.0,1,0,0,0,117600,117600,1.0,0.000,28.497215
183,-1.0,-1.0,-1.0,0,0,0,1,221952,221952,1.0,887.808,784.479304
73,-1.0,-1.0,-1.0,0,0,0,1,352800,352800,1.0,1411.200,1210.385057
6,-1.0,-1.0,-1.0,0,0,0,1,1605632,401408,4.0,1605.632,1459.475633


In [23]:
# lstm.summary()

tf.keras.backend.clear_session() # IMPORTANT for layer names to match up.

lstm = create_lstm()

# tf.keras.backend.set_learning_phase(0)
lstm_benchmark = benchmark_model(lstm)
lstm_benchmark.sample(15)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Retrieving saved model.
Retrieving saved benchmark results.


,[node type],[start],[first],[avg ms],[%],[cdf%],[mem KB],[times called],[Name],name
59,Const,190.623,0.003,0.002,0.220%,14.592%,0.000,250,lstm5/while/strided_slice_1,lstm5
241,Const,91.014,0.003,0.002,0.187%,52.812%,0.000,250,lstm3/while/clip_by_value_2/y,lstm3
231,Enter,71.851,0.006,0.009,0.004%,51.339%,0.000,1,lstm3/while/Enter_4,lstm3
218,TensorArrayV3,71.825,0.028,0.028,0.011%,48.341%,0.180,1,lstm3/TensorArray,lstm3
256,Identity,90.963,0.004,0.003,0.270%,55.872%,0.000,250,lstm3/while/Identity,lstm3
381,Const,-24.267,0.002,0.002,0.001%,75.559%,0.000,1,max_pooling1d/ExpandDims/dim,max_pooling1d
91,TensorArrayGatherV3,170.406,0.429,0.238,0.094%,19.272%,32.000,1,lstm4/TensorArrayStack/TensorArrayGatherV3,lstm4
328,Merge,46.094,0.012,0.005,0.448%,72.481%,0.004,251,lstm2/while/Merge_4,lstm2
283,Minimum,46.115,0.004,0.002,0.243%,61.384%,0.000,250,lstm2/while/clip_by_value_1/Minimum,lstm2
28,Tanh,195.105,0.004,0.002,0.241%,5.870%,0.000,250,lstm5/while/Tanh,lstm5


In [24]:
lstm_features = get_layer_features(lstm)
lstm_df = join_benchmark(lstm_features, lstm_benchmark)
cleaned_lstm = clean(lstm_df)


In [25]:
rf, X_test, y_test = rf_regression_model(cleaned_lstm, '[avg ms]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'avg_ms_actual': y_test, 'avg_ms_pred': y_pred})], axis=1)

MSE: 0.0019752402566664364


,units,filters,kernel_size,activation_relu,activation_sigmoid,activation_tanh,activation_nan,input_size,output_size,stride_size,avg_ms_actual,avg_ms_pred
7,48.0,-1.0,-1.0,0,0,1,0,32000,48,-1.0,0.941,0.896243
1,-1.0,32.0,3.0,1,0,0,0,16000,16000,1.0,0.166,0.215739
5,64.0,-1.0,-1.0,0,0,1,0,8000,16000,-1.0,0.940,0.901940


In [26]:
rf, X_test, y_test = rf_regression_model(cleaned_lstm, '[mem KB]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'mem_kb_actual': y_test, 'mem_kb_pred': y_pred})], axis=1)

MSE: 224.08000590825523


,units,filters,kernel_size,activation_relu,activation_sigmoid,activation_tanh,activation_nan,input_size,output_size,stride_size,mem_kb_actual,mem_kb_pred
7,48.0,-1.0,-1.0,0,0,1,0,32000,48,-1.0,32.0,26.336732
1,-1.0,32.0,3.0,1,0,0,0,16000,16000,1.0,64.0,39.488188
5,64.0,-1.0,-1.0,0,0,1,0,8000,16000,-1.0,32.0,38.272040


In [27]:
#lstm stacked avg ms
rf, X_test, y_test = stacked_regression_model(cleaned_lstm, '[avg ms]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'avg_ms_actual': y_test, 'avg_ms_pred': y_pred})], axis=1)

/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


MSE: 0.1608938618446589


/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

,units,filters,kernel_size,activation_relu,activation_sigmoid,activation_tanh,activation_nan,input_size,output_size,stride_size,avg_ms_actual,avg_ms_pred
7,48.0,-1.0,-1.0,0,0,1,0,32000,48,-1.0,0.941,0.546078
4,32.0,-1.0,-1.0,0,0,1,0,2500,8000,-1.0,0.941,0.514860
0,-1.0,-1.0,-1.0,0,0,0,1,500,16000,-1.0,0.033,0.413949


In [28]:
#lstm stacked mem
rf, X_test, y_test = stacked_regression_model(cleaned_lstm, '[mem KB]')

y_pred = rf.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

pd.concat([X_test, pd.DataFrame({'mem_kb_actual': y_test, 'mem_kb_pred': y_pred})], axis=1)

/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


MSE: 144.29697888671572


/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/brian/anaconda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

,units,filters,kernel_size,activation_relu,activation_sigmoid,activation_tanh,activation_nan,input_size,output_size,stride_size,mem_kb_actual,mem_kb_pred
3,10.0,-1.0,-1.0,0,0,1,0,8000,2500,-1.0,32.0,33.684898
0,-1.0,-1.0,-1.0,0,0,0,1,500,16000,-1.0,64.0,43.805198
2,-1.0,-1.0,-1.0,0,0,0,1,16000,8000,2.0,32.0,27.285977


In [30]:
# lstm.summary()

tf.keras.backend.clear_session() # IMPORTANT for layer names to match up.

enc, dec = create_nmt_models()

# tf.keras.backend.set_learning_phase(0)
enc_benchmark = benchmark_model(enc)
enc_benchmark.sample(15)

Saving model...
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.


AttributeError: Layer encoder is not connected, no input to return.

In [ ]:
dec_benchmark = benchmark_model(dec)
dec_benchmark.sample(15)